<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/87_plane_crashes_geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [ ]:
%%capture
!pip install osmnx

In [ ]:
from functools import cache

import geopandas as gpd
import osmnx as ox
import pandas as pd
import pooch
from rich.progress import track
from shapely.geometry import Point

In [ ]:
plane_crashes_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/table/plane_crashes_raw.csv',
    known_hash='265b3e905741f10c1f261c8bff0c5a19d5b0181313dfca33eb12a7b5a7916de7'
)

In [ ]:
plane_crashes = (
    pd.read_csv(plane_crashes_file)
    .dropna(subset=['Location', 'Route'])
    .query('Route.str.contains("-")')
    .query('~Aboard.str.contains("\?")')
    .assign(
        Aboard_Count=lambda df_: df_.Aboard.str.split('(').str[0].astype(float),
        Fatalities_Count=lambda df_: df_.Fatalities.str.split('(').str[0].astype(float),
        From=lambda df_: df_.Route.str.split(' - ').str[0],
        To=lambda df_: df_.Route.str.split(' - ').str[1],
    )
    .reset_index(drop=True)
)

In [ ]:
@cache
def geocode(x):
    try:
        return Point(ox.geocode(x)[::-1])
    except:
        return None


for ix, rec in track(plane_crashes.iterrows(), total=len(plane_crashes)):
    plane_crashes.loc[ix, 'From_pt'] = geocode(rec.From)
    plane_crashes.loc[ix, 'To_pt'] = geocode(rec.To)

Output()

In [ ]:
plane_crashes = gpd.GeoDataFrame(plane_crashes)
plane_crashes.to_file('plane_crashes.gpkg')

In [ ]:
plane_crashes.shape

(3981, 19)

In [ ]:
print(plane_crashes.sample(5).to_string())

                   Date  Time                            Location                             Operator Flight #                          Route                    AC  Type Registration      cn / ln                      Aboard                 Fatalities Ground                                                                                                                                                                                                                                                                                                                                                                                                                    Summary  Aboard_Count  Fatalities_Count                   From           To                                       From_pt                           To_pt
154   November 25, 1932     ?                  Marianna, Arkansas                       Hunter Airways        ?  Little Rock, AR - Memphis, TN             Travel Air 6000       NC369M 